In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rename_distr(data):
    data.distribution[data.distribution=='Agent_based_normal'] = 'ABN'
    data.distribution[data.distribution=='Agent_based_uniform'] = 'ABU'
    data.distribution[data.distribution=='F_distribution'] = 'F'
    data.distribution[data.distribution=='Laplace_or_double_exponential'] = 'Laplace'# r'$\beta$'
    data.distribution[data.distribution=='Modified_uniform_distribution'] = 'MU'
    data.distribution[data.distribution=='Normal_distribution'] = r'Normal'
    data.distribution[data.distribution=='Rayleigh_distribution'] = 'Rayleigh'
    data.distribution[data.distribution=='SVA_BETA_distribution'] = r'SVA-$\beta$'
    data.distribution[data.distribution=='Weibull_distribution'] = 'Weibull'
    data.distribution[data.distribution=='Weighted_random_with_chisquare'] = 'WRC'
    return data

## Summarize Quantum Annealing Results

### 2 and 3 agents

In [ ]:
filepath=os.path.join("output","12","QAOA_QA_23", "all_results.csv")
full_data=pd.read_csv(filepath)

data = full_data[(full_data.device =="dwave") & (full_data.flag==True) ]
data = data.sort_values('rank', axis=0).drop_duplicates(subset=['distribution', 'n_agents'], keep='first')
data = data.drop(['p', 'time_bilp', 'flag'], axis = 1)
data_23 = data.sort_values(['distribution', 'n_agents'], axis=0).reset_index(drop=True)

### From 4 to 7 agents

In [ ]:
folder = os.path.join('output', '12', 'QA_hyper_params_47')
file = 'all_results.csv'

current_folder = folder + f'_{3}'
file_path = os.path.join(current_folder, file)

full_data = pd.read_csv(file_path)

for i in range(4,8):
    current_filepath = os.path.join(folder + f'_{i}', file)
    full_data = full_data.append(pd.read_csv(current_filepath))
full_data = full_data.drop(['p', 'time_bilp', 'flag'], axis = 1)

In [ ]:
data_47 = full_data.sort_values('rank', axis=0).drop_duplicates(subset=['distribution', 'n_agents'], keep='first')
data = data_23.append(data_47)
data = data.sort_values(['distribution', 'n_agents'])

data = rename_distr(data)
data.reset_index()

In [ ]:
decoded_solutions = []
for row in data.iterrows():
    solution = row[1]['solution']
    if ',' not in solution: solution = solution.replace('.',',')
    solution = ast.literal_eval(solution)
    temp = []
    for y,qubit in enumerate(solution):
        y+=1
        if int(qubit):
            temp.append(','.join([str(idx+1) for idx,bit in enumerate(bin(y)[2:][::-1]) if int(bit)]))
    decoded_solutions.append(temp)
data['decoded_solutions'] = decoded_solutions

In [ ]:
data.to_csv(os.path.join("output","12","QA_results_decoded.csv"))
data

## Collect results from QAOA

In [ ]:
filepath=os.path.join("output","12","QAOA_QA_23", "all_results.csv")
full_data=pd.read_csv(filepath)

data = full_data[(full_data.device =="QAOA") & (full_data.flag==True) ]
data = data.sort_values('rank', axis=0).drop_duplicates(subset=['distribution', 'n_agents'], keep='first')

In [ ]:
filepath=os.path.join("output","12","QAOA_QA_23", "all_results.csv")
full_data=pd.read_csv(filepath)
full_data

In [ ]:
import ast
import glob
path = os.path.join("output","12","QAOA_QA_23")
csvfiles = glob.glob(path + "/**/all_results.csv", recursive = True)

for filepath in csvfiles:
    full_data=pd.read_csv(filepath) 
    decoded_solutions = []
    for row in full_data.iterrows():
        if row[1]['flag']:
            solution = row[1]['solution']
            if ',' not in solution: solution = solution.replace('.',',')
            solution = ast.literal_eval(solution)
            temp = []
            for y,qubit in enumerate(solution):
                y+=1
                if int(qubit):
                    temp.append(','.join([str(idx+1) for idx,bit in enumerate(bin(y)[2:][::-1]) if int(bit)]))
            decoded_solutions.append(temp)
        else:
            decoded_solutions.append(None)
    full_data['decoded_solutions'] = decoded_solutions
full_data

In [ ]:
full_data.to_csv(os.path.join("output","12", "QAOA_results_decoded.csv"))